# **CSI 382 - Data Mining and Knowledge Discovery**

# **Lab 9 - Model Evaluation Techniques**

Nestled between the modeling and deployment phases comes the crucial evaluation phase, techniques for which are discussed in this Lecture. By the time we
arrive at the evaluation phase, the modeling phase has already generated one or
more candidate models. It is of critical importance that these models be evaluated for quality and effectiveness before they are deployed for use in the field.

Deployment of data mining models usually represents a capital expenditure and
investment on the part of the company. If the models in question are invalid, the
company’s time and money are wasted.

In [ ]:
# Import the packages

import pandas as pd
import numpy as np
import itertools

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import matplotlib.gridspec as gridspec
import seaborn as sns
sns.set(style='whitegrid', palette='muted', font_scale=1.5)
from pylab import rcParams
rcParams['figure.figsize'] = 12, 8
import os

# Any results you write to the current directory are saved as output.

# **Dataset for Lab 9**

**Data Set Information:**

This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset. Several constraints were placed on the selection of these instances from a larger database. In particular, all patients here are females at least 21 years old of Pima Indian heritage.

**Content**

The datasets consists of several medical predictor variables and one target variable, Outcome. Predictor variables includes the number of pregnancies the patient has had, their BMI, insulin level, age, and so on.

**Problem Statement**

Can we build a machine learning model to accurately predict whether or not the patients in the dataset have diabetes or not?


**Acknowledgement**

Smith, J.W., Everhart, J.E., Dickson, W.C., Knowler, W.C., & Johannes, R.S. (1988). Using the ADAP learning algorithm to forecast the onset of diabetes mellitus. In Proceedings of the Symposium on Computer Applications and Medical Care (pp. 261--265). IEEE Computer Society Press.


The dataset can be found here in this [URL](https://drive.google.com/file/d/13AMz4DPhijR7rPSTq8ih5Mg_ovh3a1TR/view?usp=sharing)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## **Loading the dataset**

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/diabetes1.csv')

#Check number of rows and columns in the dataset
print("The dataset has %d rows and %d columns." % df.shape)

# **Data Preprocessing**

Checking for null values

In [ ]:
df.isnull().sum()

Describing the statistical inference from our dataset

In [ ]:
df.describe()

In [ ]:
df.info()

# **Exploring the dataset**

Outcome is the target column in the dataset. Outcome '1' is with diabetes and '0' is without diabetes, we get their total counts in the dataset. Other columns in the dataset will be input to the models. Let us have a look at the count of Outcome columns.

In [ ]:
sns.countplot(x='Outcome',data=df)
plt.show()

Basic stat summary of the feature columns against the Outcome column to see any differences between having and not having diabetes.
We can notice some mean differences between having diabetes and no diabetes (may not be statistically significant within certain confidence level). It will be difficult to judge though if you have many more columns.

In [ ]:
grouped = df.groupby('Outcome').agg({'Pregnancies':['mean', 'std', min, max],
                                       'Glucose':['mean', 'std', min, max],
                                       'BloodPressure':['mean', 'std', min, max],
                                       'SkinThickness':['mean', 'std', min, max],
                                       'Insulin':['mean', 'std', min, max],
                                       'BMI':['mean', 'std', min, max],
                                       'DiabetesPedigreeFunction':['mean', 'std', min, max],
                                       'Age':['mean', 'std', min, max]
                                      })
grouped.columns = ["_".join(x) for x in grouped.columns.ravel()]
grouped # or grouped.T

Let us have look at the distribution of the features grouping them by the Outcome column.
Outcome 0 is no diabetes, 1 is with diabetes.

In [ ]:
plt.subplots_adjust(top=5)
columns=df.columns[:8]
plt.figure(figsize=(12,28*4))
gs = gridspec.GridSpec(28, 1)
for i, cn in enumerate(df[columns]):
    ax = plt.subplot(gs[i])
    sns.distplot(df[cn][df.Outcome == 1], bins=50)
    sns.distplot(df[cn][df.Outcome == 0], bins=50)
    ax.set_xlabel('')
    plt.legend(df["Outcome"])
    ax.set_title('histogram of feature: ' + str(cn))
plt.show()

Distribution of diabetes cases are overall similar to distribution of non-diabetes distribution in each feature. No single parameter can explain the difference betweeen diabetes and non-diabetes.

In [ ]:
sns.heatmap(df[df.columns[:8]].corr(),annot=True,cmap='RdYlGn')
fig=plt.gcf()
plt.show()

Input parameters are not highly correlated. So we can keep all the parameters for the model. Another way to look at it is by doing pair plots.

# **Model Building**

In [ ]:
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

Split into train and test stratifying the 'Outcome' column.
This guarantees the train-test split ratio in the 'Outcome' column both in the training set and the testing set.

# **Methods for evaluating the performance of a Classifier**


## **Holdout Method**

In the holdout method, the original data with labeled examples is partitioned into
two disjoint sets, called the training and the test sets, respectively. A classifica-
tion model is then induced from the training set and its performance is evaluated
on the test set. The proportion of data reserved for training and for testing is
typically at the discretion of the analysts (e.g., 50-50 or two- thirds for training
and one-third for testing). The accuracy of the classifier can be estimated based
on the accuracy of the induced model on the test set.

In [ ]:
outcome=df['Outcome']
data=df[df.columns[:8]]

train,test = train_test_split(df,test_size=0.25,random_state=0,stratify=df['Outcome'])# stratify the outcome

train_X=train[train.columns[:8]]
test_X=test[test.columns[:8]]
train_Y=train['Outcome']
test_Y=test['Outcome']

In [ ]:
print(train_X.shape)
print(test_X.shape)

### **Feature Centering and Scaling**

train_X and test_X datasets are centered to zero and normalized by the std dev. This helps in faster gradient descent.


In [ ]:
features = train_X.columns.values

for feature in features:
    mean, std = df[feature].mean(), df[feature].std()
    train_X.loc[:, feature] = (train_X[feature] - mean) / std
    test_X.loc[:, feature] = (test_X[feature] - mean) / std

### **Compare model accuracies**

In [ ]:
accuracy_scores=[]

classifiers=['Linear Svm','Radial Svm','Logistic Regression','KNN','Decision Tree', 'Random forest', 'Naive Bayes']

models=[svm.SVC(kernel='linear'),svm.SVC(kernel='rbf'),LogisticRegression(),
        KNeighborsClassifier(n_neighbors=3),DecisionTreeClassifier(),
        RandomForestClassifier(n_estimators=100,random_state=0), GaussianNB()]

for i in models:
    model = i
    model.fit(train_X,train_Y)
    prediction=model.predict(test_X)
    accuracy_scores.append(metrics.accuracy_score(prediction,test_Y))

models_dataframe=pd.DataFrame(accuracy_scores,index=classifiers)
models_dataframe.columns=['Accuracy']
models_dataframe.sort_values(['Accuracy'], ascending=False)

We see that random forest classifier has better accuracy of 80.72%. Next, Logistic Regression, Radial SVM, Decision Tree and Linear SVM has similar accuracy of more than 77%. Let us look at the feature importance in Random forest classifier.

In [ ]:
modelRF= RandomForestClassifier(n_estimators=100,random_state=0)
modelRF.fit(train_X,train_Y)
predictionRF=modelRF.predict(test_X)
pd.Series(modelRF.feature_importances_,index=train_X.columns).sort_values(ascending=False)

Top five important features are 'Glucose', 'BMI', 'Age', 'DiabetesPedigreeFunction' and 'Blood pressure'. Next, pregnancies (only in case of women) have higher chances of diabetes.

## **k-fold Cross validation**

An alternative to random subsampling is cross-validation. In this approach, each
record is used the same number of times for training and exactly once for test-
ing. To illustrate this method, suppose we partition the data into two equal-sized
subsets. First, we choose one of the subsets for training and the other for testing.
We then swap the roles of the subsets so that the previous training set becomes
the test set and vice versa. This approach is called a two- fold cross-validation.
The total error is obtained by summing up the errors for both runs. In this exam-
ple, each record is used exactly once for training and once for testing.



Most of the time, the dataset has imbalance in classes, say one or more class(es) has higher instances than the other classes. In that case, we should train the model in each and every instances of the dataset. After that we take average of all the noted accuracies over the dataset.

For a K-fold cross validation, we first divide the dataset into K subsets.
Then we train on K-1 parts and test on the remainig 1 part.
We continue to do that by taking each part as testing and training on the remaining K-1. Accuracies and errors are then averaged to get an average accuracy of the algorithm.

For certain subset the algorithm will underfit while for a certain other it will overfit. With K-fold cross validation we achieve a generalized model.

Below we approach the K-fold cross validation for this dataset.


In [ ]:
from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.preprocessing import StandardScaler #Standardisation

In [ ]:
kfold = KFold(n_splits=10, random_state=42, shuffle=True) # k=10 splits the data into 10 equal parts

In [ ]:
# Starting with the original dataset and then doing centering and scaling
features=df[df.columns[:8]]
features_standard=StandardScaler().fit_transform(features)# Gaussian Standardisation
X=pd.DataFrame(features_standard,columns=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',
                                          'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age'])
X['Outcome']=df['Outcome']

In [ ]:
xyz=[]
accuracy=[]
classifiers=['Linear Svm','Radial Svm','Logistic Regression','KNN','Decision Tree', 'Random forest', 'Naive Bayes']
models=[svm.SVC(kernel='linear'),svm.SVC(kernel='rbf'),LogisticRegression(),
        KNeighborsClassifier(n_neighbors=3),DecisionTreeClassifier(),
        RandomForestClassifier(n_estimators=100,random_state=0), GaussianNB()]

for i in models:
    model = i
    cv_result = cross_val_score(model,X[X.columns[:8]], X['Outcome'], cv = kfold, scoring = "accuracy")
    xyz.append(cv_result.mean())
    accuracy.append(cv_result)

cv_models_dataframe=pd.DataFrame(xyz, index=classifiers)
cv_models_dataframe.columns=['CV Mean']
cv_models_dataframe
cv_models_dataframe.sort_values(['CV Mean'], ascending=False)

# **Comparing data mining methods**

We often need to compare two different learning methods on the same problem
to see which is the better one to use. It seems simple: estimate the error using
cross-validation (or any other suitable estimation procedure), perhaps repeated
several times, and choose the scheme whose estimate is smaller. This is quite
sufficient in many practical applications: if one method has a lower estimated
error than another on a particular dataset, the best we can do is to use the former
method’s model.

However, it may be that the difference is simply caused by estimation error,
and in some circumstances it is important to determine whether one scheme is
really better than another on a particular problem. This is a standard challenge
for machine learning researchers. If a new learning algorithm is proposed, its
proponents must show that it improves on the state of the art for the problem at
hand and demonstrate that the observed improvement is not just a chance effect
in the estimation process.

In [ ]:
box=pd.DataFrame(accuracy,index=[classifiers])
boxT = box.T

In [ ]:
ax = sns.boxplot(data=boxT, orient="h", palette="Set2", width=.6)
ax.set_yticklabels(classifiers)
ax.set_title('Cross validation accuracies with different classifiers')
ax.set_xlabel('Accuracy')
plt.show()

The above plots shows that Linear SVM, Radial SVM and Logistic Regression performs better in cross validation while tree based methods, Decision Tree and Random Forrest are worse with wider distribution of their acccuracies.

# **Ensembling**

In ensemble methods, we create multiple models and then combine them that gives us better results. Enseble methods typically gives better accuracy than a single model. The models used to create such ensemble models are called base models.

Let us do ensembling with Voting Ensemble. First we create two or more standalone models on the training dataset. A voting classifier wrap the models to get average predictions. Models with higher individual accuracies are weighted higher.

Since KNN, Decision Tree and Random Forest models have wide range of accuracies in K-fold validation, they are not considered in ensembling the models.

Other the models: Linear SVM, Radial SVM and Logistic Regression are combined together to get an ensemble model.

In [ ]:
linear_svm=svm.SVC(kernel='linear',C=0.1,gamma=10, probability=True)
radial_svm=svm.SVC(kernel='rbf',C=0.1,gamma=10, probability=True)
lr=LogisticRegression(C=0.1)

In [ ]:
from sklearn.ensemble import VotingClassifier #for Voting Classifier

## **Ensamble with 3 classifiers combined: Linear SVM, radial SVM, Log Reg**

In [ ]:
ensembleModel=VotingClassifier(estimators=[('Linear_svm',linear_svm), ('Radial_svm', radial_svm), ('Logistic Regression', lr)],
                                            voting='soft', weights=[3,1,2])

ensembleModel.fit(train_X,train_Y)
predictEnsemble = ensembleModel.predict(test_X)

In [ ]:
print('Accuracy of ensembled model with all the 3 classifiers is:', np.round(ensembleModel.score(test_X,test_Y), 4))

# **ROC curve with AUC**

Lift charts are a valuable tool, widely used in marketing. They are closely re-
lated to a graphical technique for evaluating data mining schemes known as ROC
curves, which are used in just the same situation as the preceding one, in which
the learner is trying to select samples of test instances that have a high propor-
tion of positives. The acronym stands for receiver operating characteristic, a
term used in signal detection to characterize the trade-off between hit rate and
false alarm rate over a noisy channel.
ROC curves depict the performance of a classifier without regard to class distri-
bution or error costs.

In [ ]:
from sklearn import metrics
from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc, roc_curve, recall_score,
                             classification_report, f1_score, average_precision_score, precision_recall_fscore_support)

### **For the ensembling method**

In [ ]:
from sklearn.metrics import roc_curve, auc
false_positive_rate, true_positive_rate, thresholds = roc_curve(test_Y, predictEnsemble)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
plt.title('Receiver Operating Characteristic for ensembling method')
plt.plot(false_positive_rate,true_positive_rate, color='red',label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],linestyle='--')
plt.axis('tight')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

### **For all methods**

In [ ]:
# Logistic regression
modelLR = LogisticRegression()
modelLR.fit(train_X,train_Y)
y_pred_prob_lr = modelLR.predict_proba(test_X)[:,1]
fpr_lr, tpr_lr, thresholds_lr = roc_curve(test_Y, y_pred_prob_lr)
roc_auc_lr = auc(fpr_lr, tpr_lr)
precision_lr, recall_lr, th_lr = precision_recall_curve(test_Y, y_pred_prob_lr)

# SVM with rbf
modelSVMrbf=svm.SVC(kernel='rbf', probability=True)
modelSVMrbf.fit(train_X,train_Y)
y_pred_prob_SVMrbf = modelSVMrbf.predict_proba(test_X)[:,1]
fpr_SVMrbf, tpr_SVMrbf, thresholds_SVMrbf = roc_curve(test_Y, y_pred_prob_SVMrbf)
roc_auc_SVMrbf = auc(fpr_SVMrbf, tpr_SVMrbf)
precision_SVMrbf, recall_SVMrbf, th_SVMrbf = precision_recall_curve(test_Y, y_pred_prob_SVMrbf)

# SVM with linear
modelSVMlinear=svm.SVC(kernel='linear', probability=True)
modelSVMlinear.fit(train_X,train_Y)
y_pred_prob_SVMlinear = modelSVMlinear.predict_proba(test_X)[:,1]
fpr_SVMlinear, tpr_SVMlinear, thresholds_SVMlinear = roc_curve(test_Y, y_pred_prob_SVMlinear)
roc_auc_SVMlinear = auc(fpr_SVMlinear, tpr_SVMlinear)
precision_SVMlinear, recall_SVMlinear, th_SVMlinear = precision_recall_curve(test_Y, y_pred_prob_SVMlinear)

# KNN
modelKNN = KNeighborsClassifier(n_neighbors=3)
modelKNN.fit(train_X,train_Y)
y_pred_prob_KNN = modelKNN.predict_proba(test_X)[:,1]
fpr_KNN, tpr_KNN, thresholds_KNN = roc_curve(test_Y, y_pred_prob_KNN)
roc_auc_KNN = auc(fpr_KNN, tpr_KNN)
precision_KNN, recall_KNN, th_KNN = precision_recall_curve(test_Y, y_pred_prob_KNN)


# Decision Tree
modelTree=DecisionTreeClassifier()
modelTree.fit(train_X,train_Y)
y_pred_prob_Tree = modelTree.predict_proba(test_X)[:,1]
fpr_Tree, tpr_Tree, thresholds_Tree = roc_curve(test_Y, y_pred_prob_Tree)
roc_auc_Tree = auc(fpr_Tree, tpr_Tree)
precision_Tree, recall_Tree, th_Tree = precision_recall_curve(test_Y, y_pred_prob_Tree)

# Random forest
modelRF= RandomForestClassifier(n_estimators=100,random_state=0)
modelRF.fit(train_X,train_Y)
y_pred_prob_rf = modelRF.predict_proba(test_X)[:,1]
fpr_rf, tpr_rf, thresholds_rf = roc_curve(test_Y, y_pred_prob_rf)
roc_auc_rf = auc(fpr_rf, tpr_rf)
precision_rf, recall_rf, th_rf = precision_recall_curve(test_Y, y_pred_prob_rf)


# Naive Bayes
modelNB= GaussianNB()
modelNB.fit(train_X,train_Y)
y_pred_prob_nb = modelNB.predict_proba(test_X)[:,1]
fpr_nb, tpr_nb, thresholds_nb = roc_curve(test_Y, y_pred_prob_nb)
roc_auc_nb = auc(fpr_nb, tpr_nb)
precision_nb, recall_nb, th_nb = precision_recall_curve(test_Y, y_pred_prob_nb)

# Ensemble
y_pred_prob_en = ensembleModel.predict_proba(test_X)[:,1]
fpr_en, tpr_en, thresholds_en = roc_curve(test_Y, y_pred_prob_en)
roc_auc_en = auc(fpr_en, tpr_en)
precision_en, recall_en, th_en = precision_recall_curve(test_Y, y_pred_prob_en)

In [ ]:
# Plot ROC curve
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_lr, tpr_lr, label='Log Reg (area = %0.3f)' % roc_auc_lr)
plt.plot(fpr_SVMrbf, tpr_SVMrbf, label='SVM rbf (area = %0.3f)' % roc_auc_SVMrbf)
plt.plot(fpr_SVMlinear, tpr_SVMlinear, label='SVM linear (area = %0.3f)' % roc_auc_SVMlinear)
plt.plot(fpr_KNN, tpr_KNN, label='KNN (area = %0.3f)' % roc_auc_KNN)
plt.plot(fpr_Tree, tpr_Tree, label='Tree (area = %0.3f)' % roc_auc_Tree)
plt.plot(fpr_rf, tpr_rf, label='RF (area = %0.3f)' % roc_auc_rf)
plt.plot(fpr_nb, tpr_nb, label='NB (area = %0.3f)' % roc_auc_nb)
plt.plot(fpr_en, tpr_en, label='Ensamble (area = %0.3f)' % roc_auc_en)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curves from the investigated models')
plt.legend(loc='best')
plt.show()

We see that Tree, KNN, RF and NB models are somewhat worse compared to the other models in AUC.
Ensamble model is not standing out compared to LogReg and SVM models.
Nevertheless, ensamble model is expected to be more robust for any future addition of training data.

## **Precision-recall curve comparing the models**

Information retrieval researchers define parameters called *recall* and *precision*:
\begin{equation*}
    \begin{split}
        {recall} &= \frac{{number of documents retrieved that are relevant}}{{total number of documents that are relevant}}\\
        {precision} &= \frac{{number of documents retrieved that are relevant}}{{total number of documents that are retrieved }}
    \end{split}
\end{equation*}


In [ ]:
plt.plot([1, 0], [0, 1], 'k--')
plt.plot(recall_lr, precision_lr, label='Log Reg')
plt.plot(recall_SVMrbf, precision_SVMrbf, label='SVM rbf')
plt.plot(recall_SVMlinear, precision_SVMlinear, label='SVM linear')
plt.plot(recall_KNN, precision_KNN, label='KNN')
plt.plot(recall_Tree, precision_Tree, label='Tree')
plt.plot(recall_rf, precision_rf, label='RF')
plt.plot(recall_nb, precision_nb, label='NB')
plt.plot(recall_en, precision_en, label='Ensamble')
plt.title('Precision vs. Recall')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend(loc='best')
plt.show()

With higher recall (outcome = 1 i.e. having diavetes), precision (outcome=0, i.e. no diabetes) goes down.
Again the ensamble model is comparable to LogReg and SVM models and other models are worse.


# **Predictive Outcomes**

## **Confusion matrix with ensemble model**

Let us look at the confusion matrix from the ensamble classifier.
First define a function to plot confusion matrix

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In the two-class case with classes yes and no, lend or not lend, mark a suspicious
patch as an oil slick or not, and so on, a single prediction has the four different
possible outcomes shown in Figure 1.
The true positives (TP) and true negatives (TN) are correct classifications.
A false positive (FP) occurs when the outcome is incorrectly predicted as yes
(or positive) when it is actually no (negative).
A false negative (FN) occurs when the outcome is incorrectly predicted as neg-
ative when it is actually positive.

In [ ]:
class_names = test_Y.unique()
cmEnsamble = confusion_matrix(test_Y, predictEnsemble)
plt.grid(False)
plot_confusion_matrix(cmEnsamble, classes=class_names, title='Confusion matrix with ensamble model, without normalization')


The true positive rate is TP divided by the total number of positives, which is
TP + FN;
the false positive rate is FP divided by the total number of negatives, FP +
TN.
The overall success rate is the number of correct classifications divided by the
total number of classifications:
TP+TN
TP+TN+FP+FN
Finally, the error rate is one minus this.

## **Classification Report**

In [ ]:
print(metrics.classification_report(test_Y, predictEnsemble))

Looking at the recall scores for 0 (no diabetes) and 1 (having diabetes), the model did well for predicting no diabetes (correct 92% of the time) while did not do that well for predicting diabetes (correct 54% of the time). So, there is room for improving prediction for diabetes.

# **Cost-sensitive learning**

## **Diabetes prediction using Neural Network with Keras**

Keras is a high level frame work for running neural network applications. It runs tensorflow at the backend.

Suppose that you artificially increase the number of no instances by a factor of
10 and use the resulting dataset for training. If the learning scheme is striving to
minimize the number of errors, it will come up with a decision structure that is
biased toward avoiding errors on the no instances, because such errors are effec-
tively penalized 10-fold. If data with the original proportion of no instances is
used for testing, fewer errors will be made on these than on yes instances—that
is, there will be fewer false positives than false negatives—because false posi-
tives have been weighted 10 times more heavily than false negatives.

Varying the proportion of instances in the training set is a general technique
for building cost-sensitive classifiers.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau

# fix random seed for reproducibility
np.random.seed(42)

In [ ]:
train_Y = to_categorical(train_Y, num_classes = 2)
test_Y = to_categorical(test_Y, num_classes = 2)

In [ ]:
# Confirm the train-test split ratio
print(np.shape(train_X))
print(np.shape(train_Y))
print(np.shape(test_X))
print(np.shape(test_Y))

## **1. Create the model using Keras**

input_dim = 8 since we have 8 input variable.
Here we are using 5 fully connected layers defined by using the Dense class (no particular reason for 5 layers, typically more layers are better). Number of neurons in the layers are the first argument (8, 12, 12, 8, 4 & 1 respecitvely here).
We use the default weight initialization in Keras which is between 0 to 0.05 assuming "uniform" distribution.
First 4 layers have "relu" activation and output layer has "sigmoid" activation. Sigmoid on the output layer ensures that we have output between 0 and 1.

We can piece it all together by adding each layer.


In [ ]:
# A regularizer that applies a L2 regularization penalty.
# The L2 regularization penalty is computed as: loss = l2 * reduce_sum(square(x))
# Try the model with and without the regularizer
from tensorflow.keras import regularizers
model = Sequential()
model.add(Dense(8, input_dim=8, activation='relu'))
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='relu'))
#model.add(Dense(2,activation='sigmoid))
model.add(Dense(2,kernel_regularizer=regularizers.L1L2(l1=1e-5,l2=1e-4), activation='sigmoid'))

In [ ]:
model.summary()

## **2. Compile the model**

Now the model is ready, we can compile it (using tensorflow under the hood or backend) and train it find the best weights for prediction. loss='binary_crossentropy' since the problem is binary classification.
optimizer='adam' since it is efficient and default default.
From metrics we collect the accuracy.


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## **3. Fit model**

We train the model by calling fit() on training data.
The number of iteration through the whole training datset is called "epoch". It is set to 150 (higher the better).
The number of instances that are evaluated before a weight update in the network is performed is the the batch size. It is set to 50 (relatively small, the dataset is also small).

With the model.fit(), we shall also capture the accuracy each epoch.


In [ ]:
epoch = 150
batch_size = 50

history = model.fit(train_X, train_Y, batch_size = batch_size, epochs = epoch,
          validation_data = (test_X, test_Y), verbose = 2)

We see that the model accuracy hovers around 81.94% in the testing data which is not better than ensemble model (accuracy may change slightly run-to-run). We can try larger network inclear number of epochs however, very likely this is the maximum prediction or performance of the model based on the dataset. We may also end of overfitting the training dataset. Again, note that the dataset is relatively small for neural network.

## **4. Evaluate model**

We evaluate the model on test dataset and obtain the score and accuracy.
Score is the evaluation of the loss function for a given input.


In [ ]:
score, acc = model.evaluate(test_X, test_Y)
print('Test score:', score)
print('Test accuracy:', acc)

So we get 78.125% accuracy on the test dataset with this fully connected neural network (varies little bit run to run).
This is comparable and not significantly better than ensemble method done earlier.
NN acccuracy expected to improve with larger amount of training data.

### **Training and validation curves vs. epoch**

In [ ]:
# Plot the loss and accuracy curves for training and validation vs. epochs

fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="Testing loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Testing accuracy")
legend = ax[1].legend(loc='best', shadow=True)

plt.show()

After about 30 epochs, testing loss starts to become higher than training loss and exactly opposite for accuracy: testing accuracy starts to get lower than training accuracy. Basically we are starting to overfit here.
Therefore, we get an optimal accuracy of close to 80% using this model.



### **Confusion matrix using this model**

Let us have a look at the correct and misclasssification in the confusion matrix.
I am using the below function for confusion matrix.

In [ ]:
# Predict the values from the validation dataset
Y_pred = model.predict(test_X)
# Convert predictions classes to one hot vectors
Y_pred_classes = np.argmax(Y_pred,axis = 1)
# Convert validation observations to one hot vectors
Y_true = np.argmax(test_Y,axis = 1)
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(2))
plt.grid(False)
plt.show()

# **Conclusion**

Close to 80% overall accuracy is obtained based on this dataset to predict diabetes in a person using ensemble model among different methods evalauted as well as NN model.
Non diabetes cases are predicted more accurately than diabetes cases.
NN model gave better prediction of diabetes cases than ensamble model (66% vs. 54%).
Based on Random Forest feature importance, Glucose, BMI, Age and diabetes pedigree are top causes for diabetes.
Based on parameter correlation, no given single parameter can predict diabetes.
More data, feature engineering and larger NN model is expected to improve diabetes prediction accuracy.

** With help from several Kaggle notebooks.**

# **That's all for today!**

# **Tasks**

## **Dataset**

Cardiovascular diseases (CVDs) are the number 1 cause of death globally, taking an estimated 17.9 million lives each year, which accounts for 31% of all deaths worlwide.

Heart failure is a common event caused by CVDs and this dataset contains 12 features that can be used to predict mortality by heart failure.

Most cardiovascular diseases can be prevented by addressing behavioural risk factors such as tobacco use, unhealthy diet and obesity, physical inactivity and harmful use of alcohol using population-wide strategies.

People with cardiovascular disease or who are at high cardiovascular risk (due to the presence of one or more risk factors such as hypertension, diabetes, hyperlipidaemia or already established disease) need early detection and management wherein a machine learning model can be of great help. For more information you can follow this link - [URL](https://archive.ics.uci.edu/ml/datasets/Heart+failure+clinical+records).

Download the dataset from here - [Download Link](https://drive.google.com/file/d/1TZx7MkdUOcxzjF98AIpc9sC50YcTNO-2/view?usp=sharing)


Now try to do the following:

1. Apply any number of models on the dataset
2. Compare the accuracies that are evident from each dataset
3. Find ROC, Precsion and Recall curves for each model separately
4. Find ROC, Precsion and Recall curves for all models combined and make a decision on the best model